# Load Test deployed web application

In this notebook, we test the latency of the deployed web application by sending a number of duplicate questions as asychronous requests.

In [1]:
import asyncio
import json
import random
import urllib.request
from timeit import default_timer

import aiohttp
from tqdm import tqdm
import requests
import pandas as pd
from utilities import text_to_json

/anaconda/envs/MLAKSDeployment/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
print(aiohttp.__version__) 

3.3.2


We will test our deployed service with 100 calls. We will only have 4 requests concurrently at any time. We have only deployed one pod on one node and increasing the number of concurrent calls does not really increase throughput. Feel free to try different values and see how the service responds.

In [3]:
NUMBER_OF_REQUESTS = 100  # Total number of requests
CONCURRENT_REQUESTS = 4   # Number of requests at a time

Get the IP address of our service.

In [5]:
service_json = !sudo kubectl get service azure-ml -o json
service_dict = json.loads(''.join(service_json))
app_url = service_dict['status']['loadBalancer']['ingress'][0]['ip']

In [6]:
scoring_url = 'http://{}/score'.format(app_url)
version_url = 'http://{}/version'.format(app_url)
health_url = 'http://{}/'.format(app_url)

In [7]:
!curl $health_url

Healthy

In [8]:
!curl $version_url # Reports the lightgbm version

2.1.2

In [9]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
dupes_to_score = dupes_test.iloc[:NUMBER_OF_REQUESTS,4]

In [10]:
url_list = [[scoring_url, jsontext] for jsontext in dupes_to_score.apply(text_to_json)]

In [11]:
def decode(result):
    return json.loads(result.decode("utf-8"))

In [12]:
async def fetch(url, session, data, headers):
    start_time = default_timer()
    async with session.request('post', url, data=data, headers=headers) as response:
        resp = await response.read()
        elapsed = default_timer() - start_time
        return resp, elapsed

In [13]:
async def bound_fetch(sem, url, session, data, headers):
    # Getter function with semaphore.
    async with sem:
        return await fetch(url, session, data, headers)

In [14]:
async def await_with_progress(coros):
    results=[]
    for f in tqdm(asyncio.as_completed(coros), total=len(coros)):
        result = await f
        results.append((decode(result[0]),result[1]))
    return results

In [15]:
async def run(url_list, num_concurrent=CONCURRENT_REQUESTS):
    headers = {'content-type': 'application/json'}
    tasks = []
    # create instance of Semaphore
    sem = asyncio.Semaphore(num_concurrent)

    # Create client session that will ensure we dont open new connection
    # per each request.
    async with aiohttp.ClientSession() as session:
        for url, data in url_list:
            # pass Semaphore and session to every POST request
            task = asyncio.ensure_future(bound_fetch(sem, url, session, data, headers))
            tasks.append(task)
        return await await_with_progress(tasks)

Below we run the 100 requests against our deployed service.

In [16]:
loop = asyncio.get_event_loop()
start_time = default_timer()
complete_responses = loop.run_until_complete(asyncio.ensure_future(run(url_list, num_concurrent=CONCURRENT_REQUESTS)))
elapsed = default_timer() - start_time
print('Total Elapsed {}'.format(elapsed))
print('Avg time taken {0:4.2f} ms'.format(1000*elapsed/len(url_list)))

100%|██████████| 100/100 [00:07<00:00, 12.98it/s]

Total Elapsed 7.708655123999051
Avg time taken 77.09 ms


In [17]:
# Example response
complete_responses[0]

({'result': [[[5223, 6700, 0.9056762423642606],
    [126100, 4889658, 0.5897041951019686],
    [7364150, 7364307, 0.013095757762535671],
    [19590865, 19590901, 0.009726747791608314],
    [11922383, 11922384, 0.007400362725375734],
    [5627284, 5627301, 0.006518820196008976],
    [1771786, 1771824, 0.004990243605901752],
    [24900875, 24900924, 0.004120260321857448],
    [7837456, 14853974, 0.0016726596738734054],
    [1129216, 1129270, 0.0014618991064883388],
    [237104, 1473742, 0.0009983357055414952],
    [784929, 784946, 0.000918666929043429],
    [23667086, 23667087, 0.0007934756119722436],
    [4616202, 4616273, 0.0006798101809810278],
    [750486, 750506, 0.0005087117457724936],
    [14028959, 8716680, 0.0004299633021580329],
    [14220321, 14220323, 0.00037730639500855277],
    [85992, 86014, 0.00030718180657696116],
    [695050, 695053, 0.00029608261688253765],
    [1822350, 1822769, 0.00024789271341276657],
    [10693845, 10693852, 0.00022361713661220404],
    [5767325, 5

Let's use the number of original questions to count the succesful responses.

In [18]:
no_questions = len(complete_responses[0][0]['result'][0])

In [19]:
num_succesful=[len(i[0]['result'][0]) for i in complete_responses].count(no_questions)
print('Succesful {} out of {}'.format(num_succesful, len(url_list)))

Succesful 100 out of 100


Next, we will explore the real-time scoring in an [iPyWidget app](08_Real_Time_Scoring.ipynb).